> Goal: count total word for every hour and period (morning, afternoon, evening, night)

> using normalized data which means only processing meaningful word(no stopwords)

In [1]:
import pandas as pd

#### reading all account data

In [2]:
df = pd.read_json('data/normalized_tweets.json')

#### creating dictionary for time period

#### generating hour

In [3]:
df['hour'] = df.datetime.apply(lambda dt: dt.hour)

#### turning message into word and counting them

In [4]:
df['word'] = df.message.apply(lambda m: m.split(' '))

df['total'] = df.word.apply(lambda w: len(w))

#### reselecting columns, remove unused columns

In [5]:
word_df = df[['hour', 'total']]
word_df.head(10)

,hour,total
0,22,26
1,16,26
2,11,25
3,22,10
4,20,22
5,17,21
6,13,25
7,20,22
8,19,24
9,7,9


#### generating total words for every period json file

#### generating total words for every hour json file

In [6]:
def convert_hour(hour):
    if hour < 10:
        return '0{}'.format(hour)
    
    return '{}'.format(hour)

In [7]:
hourly_total_word = pd.DataFrame(word_df.groupby('hour').agg({'total': 'sum'}))
hourly_total_word.reset_index(inplace=True)

hourly_total_word.hour = hourly_total_word.hour.apply(convert_hour)

hourly_total_word_json = hourly_total_word.to_json(orient='records')


In [9]:
hourly_total_word.sort_values(by=['total'], ascending=True)

,hour,total
2,02,11
1,01,13
5,05,36
3,03,68
0,00,87
6,06,107
4,04,110
7,07,282
12,12,318
16,16,322


In [10]:
# writting json result
with open('results/total_word_distribution.json', 'w', encoding='utf-8') as outfile:
    outfile.write(hourly_total_word_json)